In [1]:
import numpy as np
import pandas as pd
import warnings as wn
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import math
wn.filterwarnings('ignore')

# read the data
train = pd.read_csv(r'train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 11 columns):
id                        357 non-null int64
playtime_forever          357 non-null float64
is_free                   357 non-null bool
price                     357 non-null float64
genres                    357 non-null object
categories                357 non-null object
tags                      357 non-null object
purchase_date             355 non-null object
release_date              357 non-null object
total_positive_reviews    355 non-null float64
total_negative_reviews    355 non-null float64
dtypes: bool(1), float64(4), int64(1), object(5)
memory usage: 28.3+ KB


In [2]:
# 找genres这一列中的每一个unique值的出现次数 
genres_map = {}
genres_list = []
for row in train['genres']:
    arr = row.split(',')
    for i in arr:
        if i not in genres_map:
            genres_map[i] = 1
            genres_list.append(i)
        else:
            genres_map[i] += 1

genres_map = sorted(genres_map.items(), key = lambda x: x[1], reverse = True)
print(genres_map)
# print(len(genres_list))
for item in genres_list:
    train[item] = train['genres'].apply(lambda x: 1 if item in x else 0)
# train.info()
# print(genres_map)
# train.loc[train['genres'].str.contains('Action') & train['genres'].str.contains('Adventure')]['playtime_forever'].mean()
# train.loc[train['genres'].str.contains('Utilities')]['playtime_forever'].mean()


# 根据出现次数来进行genres重要性评估
bons_map = {}
for ge in genres_list:
    tmp = round(train[train['genres'].str.contains(ge)]['playtime_forever'].mean())
    print(ge, tmp)
    if tmp >= 10:
        bons_map[ge] = 3
    elif tmp >= 5:
        bons_map[ge] = 2
    elif tmp >= 4:
        bons_map[ge] = 1
# print(bons_map)
def computeGenScore(x):
    bons = 0
    for score in bons_map:
        if score in x:
            bons += bons_map[score]
    return bons
    
train['genres_score'] = train['genres'].apply(computeGenScore)
# train['genres_score'].value_counts()

for gs in train['genres_score'].unique():
    tmp = train[train['genres_score'] == gs]['playtime_forever'].mean()
#     print(gs, tmp)s
print(train[train['genres'].str.contains('Utilities')])

[('Indie', 185), ('Adventure', 184), ('Action', 171), ('RPG', 103), ('Strategy', 82), ('Simulation', 65), ('Casual', 34), ('Free to Play', 19), ('Massively Multiplayer', 14), ('Early Access', 9), ('Violent', 6), ('Gore', 5), ('Sports', 4), ('Racing', 2), ('Sexual Content', 2), ('Design & Illustration', 2), ('Audio Production', 1), ('Nudity', 1), ('Animation & Modeling', 1), ('Utilities', 1)]
Adventure 4
Casual 1
Indie 3
RPG 5
Action 4
Strategy 4
Simulation 4
Racing 1
Sports 2
Massively Multiplayer 11
Sexual Content 3
Violent 1
Free to Play 2
Early Access 1
Audio Production 0
Gore 0
Design & Illustration 4
Nudity 0
Animation & Modeling 5
Utilities 5
      id  playtime_forever  is_free   price  \
250  250          5.116667    False  1900.0   

                                                genres  \
250  Animation & Modeling,Design & Illustration,Uti...   

                                            categories  \
250  Steam Achievements,Steam Trading Cards,Steam W...   

              

In [3]:
# 是否有色情，暴力，血腥等成人元素
def is_adult(x):
    if 'Nudity' in x \
    or 'Sexual Content' in x \
    or 'Gore' in x\
    or 'Violent' in x:
        return 1
    else:
        return 0
    
train['adult_con'] = train['genres'].apply(is_adult)
# train['adult_con'].value_counts()

In [4]:

# 处理price
def price_clean(x):
    if x == 0:
        return 0
    else:
        return math.log(x)

train['price'] = train['price'].apply(price_clean)
# print(train['price'])
# print(np.max(train['price']))

In [5]:
# 找 categories 这一列中的每一个unique值的出现次数 
categories_map = {}
categories_list = []
for row in train['categories']:
    arr = row.split(',')
    for i in arr:
        if i not in categories_map:
            categories_map[i] = 1
            categories_list.append(i)
        else:
            categories_map[i] += 1

categories_map = sorted(categories_map.items(), key = lambda x: x[1], reverse = True)
# print(categories_list)
# genres_map
# print(len(categories_list))
for item in categories_list:
    train[item] = train['categories'].apply(lambda x: 1 if item in x else 0)
    
# train.info()
categories_map = {}
dislike_cate_map = []

for ge in categories_list:
    tmp = round(train[train['categories'].str.contains(ge)]['playtime_forever'].mean())
    if tmp >= 8:
        categories_map[ge] = tmp - 7

print(categories_map)
def computeCateScore(x):
    bons = 0
    for score in categories_map:
        if score in x:
            bons += categories_map[score]
    return bons
    
train['categories_score'] = train['categories'].apply(computeCateScore)
# train['categories_score'].value_counts()
# print(dislike_cate_map)
# print(train['is_dislike_cate'])

{'Steam Workshop': 1, 'Online Co-op': 1, 'Stats': 2, 'Local Multi-Player': 3, 'Remote Play on TV': 3}


In [6]:
# 找 tags 这一列中的每一个unique值的出现次数 
tags_map = {}
tags_list = []
for row in train['tags']:
    arr = row.split(',')
    for i in arr:
        if i not in tags_map:
            tags_map[i] = 1
            tags_list.append(i)
        else:
            tags_map[i] += 1

tags_map = sorted(tags_map.items(), key = lambda x: x[1], reverse = True)
# print(genres_list)
# genres_map
# print(tags_list)
for item in tags_list:
    if item not in train:
        train[item] = train['tags'].apply(lambda x: 1 if item in x else 0)
    else:
        for index, row in train.iterrows():
            if row[item] != 1:
                if item in row['tags']:
                    row[item] = 1
                else:
                    row[item] = 0
            else:
                row[item] = 1
    
# train.info()
tags_map = {}
for ge in tags_list:
    tmp = round(train[train['tags'].str.contains(ge)]['playtime_forever'].mean())
    if tmp >= 17:
        tags_map[ge] = tmp - 16

print(tags_map)
def computeTagScore(x):
    bons = 0
    for score in tags_map:
        if score in x:
            bons += tags_map[score]
    return bons
    
train['tags_score'] = train['tags'].apply(computeTagScore)
# train['tags_score'].value_counts()

{'Turn-Based Combat': 1, 'Touch-Friendly': 6, 'PvP': 1, 'Battle Royale': 4, 'Clicker': 9, 'Rome': 1}


In [7]:
# 是否免费 bool 转化为0/1
train['is_free'] = train['is_free'].apply(lambda x: 1 if x == True else 0)
one_hot_col = categories_list + tags_list + genres_list
# one_hot_col = categories_list +  genres_list

In [8]:
# 处理 total_positive_reviews 和 total_negative_reviews 中的空值
def clean_pos(x):
    if x != x:
        return 0
    else:
        return x

def clean_neg(x):
    if x != x or x == 0:
        return 1
    else:
        return x
    
train['total_positive_reviews'] = train['total_positive_reviews'].apply(clean_pos)
train['total_negative_reviews'] = train['total_negative_reviews'].apply(clean_neg)
train['reviews_level'] = train['total_positive_reviews']/train['total_negative_reviews']
# print(train[train['playtime_forever'] > 0]['total_positive_reviews'].mean())
# print(train[train['playtime_forever'] > 0]['total_negative_reviews'].mean())
# print(train[train['playtime_forever'] == 0]['total_positive_reviews'].mean())
# print(train[train['playtime_forever'] == 0]['total_negative_reviews'].mean())

In [9]:
# 根据评论反映游戏热度
def is_hot(x):
    if x >= 28000:
        return 2
    elif x >= 10000:
        return 1
    else:
        return 0

train['comment_total'] = train['total_positive_reviews'] + train['total_negative_reviews']
train['hot_level'] = train['comment_total'].apply(is_hot)

In [10]:
# 日期提取
import datetime
import time
date_map = {'Jul':'7', 'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5', \
            'Jun':'6', 'Aug':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

def clean_buy_date(x):
    if x != x:
        return -1
    arr = x.split(' ')
    if len(arr) != 3:
        return -1
    return arr[2] + '-' + date_map[arr[0]]

def full_clean_buy_date(x):
    if x != x:
        return -1
    arr = x.split(' ')
    if len(arr) != 3:
        return -1
    return arr[2] + '-' + date_map[arr[0]] + '-' + arr[1].split(',')[0]
    
def clean_realease_date(x):
    if x != x:
        return -1
    try:
        arr = x.split(',')
        if len(arr) != 2:
            return -1
        mon = -1
        for month in date_map:
            if month in arr[0]:
                mon = date_map[month]
        year = arr[1]
        return year + '-' + str(mon)
    except AttributeError:
        return -1

train['purchase_date_full_clean'] = train['purchase_date'].apply(full_clean_buy_date)
train['purchase_date_clean'] = train['purchase_date'].apply(clean_buy_date)
train['release_date_clean'] = train['release_date'].apply(clean_realease_date)
train['purchase_year'] = train['purchase_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[0]))
train['purchase_month'] = train['purchase_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[1]))
train['purchase_day'] = train['purchase_date_full_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[2]))
train['release_year'] = train['release_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[0]))
train['release_month'] = train['release_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[1]))

# 计算每一年中的每个月的平均游戏时长
total_play_time_map = {}
total_play_cnt_map = {}

sum_time = 0
sum_cnt = 0
for buy_time in train['purchase_date_clean'].unique():
    total_play_time_map[buy_time] = train[train['purchase_date_clean'] == buy_time]['playtime_forever'].sum()
    total_play_cnt_map[buy_time] = len(train[train['purchase_date_clean'] == buy_time])
    sum_time += train[train['purchase_date_clean'] == buy_time]['playtime_forever'].sum()
    sum_cnt += len(train[train['purchase_date_clean'] == buy_time])

time_mean = sum_time/len(train['purchase_date_clean'].unique())
cnt_mean = sum_cnt/len(train['purchase_date_clean'].unique())
print(time_mean, cnt_mean)
def month_total_play_time(x):
    if x in total_play_time_map:
        return total_play_time_map[x]
    else:
        return time_mean
    
def month_total_play_cnt(x):
    if x in total_play_time_map:
        return total_play_cnt_map[x]
    else:
        return cnt_mean

train['month_total_play_time'] = train['purchase_date_clean'].apply(month_total_play_time)
train['month_total_play_cnt'] = train['purchase_date_clean'].apply(month_total_play_cnt)
# print(train['month_total_play_cnt'])
# # 游戏总时长统计
# for year in train['purchase_year'].unique():
#     tmp =  train[train['purchase_year'] == year]
#     for month in train['purchase_month'].unique():
#         print(year, '-', month, ': ', tmp[tmp['purchase_month'] == month]['playtime_forever'].value_counts())
#         print(year, '-', month, ': ', tmp[tmp['purchase_month'] == month]['playtime_forever'].mean())
#         print(year, '-', month, ': ', tmp[tmp['purchase_month'] == month]['playtime_forever'].sum())
#         print(' ')
# print(train['purchase_day'].value_counts())

28.552991452991453 9.153846153846153


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

# final_train = pd.concat([\
#                          train[one_hot_col],\
#                          train[['total_positive_reviews', 'total_negative_reviews', 'reviews_level', 'is_free',\
#                                 'comment_total', 'hot_level', 'price', 'playtime_forever', 'adult_con',\
#                                 'month_total_play_time', 'month_total_play_cnt', 'purchase_year', 'purchase_month',\
#                                 'release_year', 'release_month', 'genres_score', 'categories_score', 'tags_score',\
#                                 ]]],\
#                         axis=1, join='inner')
final_train = train[['total_positive_reviews', 'total_negative_reviews', 'reviews_level', 'is_free',\
                    'comment_total', 'hot_level', 'price', 'playtime_forever', 'adult_con', \
                    'month_total_play_time', 'month_total_play_cnt', 'purchase_year', 'purchase_month',\
                    'release_year', 'release_month', 'genres_score', 'categories_score', 'tags_score']]

final_train.to_csv('final_train.csv', index=False)
train_x = final_train.drop(['playtime_forever'], axis=1).as_matrix()
# train_x = final_train[final_train['is_try'] == 1].drop(['playtime_forever', 'is_try'], axis=1).as_matrix()
train_y = final_train[['playtime_forever']].as_matrix()
# train_y = final_train[final_train['is_try'] == 1]['playtime_forever'].as_matrix()
# print(train_x, train_y)
# pipe=Pipeline('select',SelectKBest(k=20))

# param_test = {'classify__n_estimators':list(range(20,50,2)), 
#               'classify__max_depth':list(range(3,60,3))}
# gsearch = GridSearchCV(estimator = pipe, param_grid = param_test, cv=10)
# gsearch.fit(train_x,train_y)

# print(gsearch.best_params_, gsearch.best_score_)


In [12]:
rfr=RandomForestRegressor()
rfr.fit(train_x, train_y)

model_save_path = "C:/Users/XHDMW/Desktop/ML/5001 Kaggle/msbd5001-fall2019/model_save/train_model.m"
print("HR Model save...")

joblib.dump(rfr, model_save_path)
print("Model has been saved.")

HR Model save...
Model has been saved.


In [13]:
# from sklearn.metrics import mean_squared_error
# rfr=RandomForestRegressor(random_state=10)
 
# X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.33, random_state=0)


# rfr.fit(X_train,y_train)   #训练数据

# prediction = rfr.predict(X_test)
# print('MSE=', mean_squared_error(prediction, y_test))
# error = []
# for i in range(len(y_test)):
#     error.append(y_test[i] - prediction[i])

# squaredError = []
# absError = []
# for val in error:
#     squaredError.append(val * val)#target-prediction之差平方 
#     absError.append(abs(val))#误差绝对值

# print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#均方根误差RMSE



# rfr=RandomForestRegressor(n_estimators = 50, max_depth = 7, min_samples_split = 170, random_state=10)
 
# X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.33, random_state=10)


# rfr.fit(X_train,y_train)   #训练数据

# prediction = rfr.predict(X_test)
# print('MSE=', mean_squared_error(prediction, y_test))

# error = []
# for i in range(len(y_test)):
#     error.append(y_test[i] - prediction[i])

# squaredError = []
# absError = []
# for val in error:
#     squaredError.append(val * val)#target-prediction之差平方 
#     absError.append(abs(val))#误差绝对值

# print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#均方根误差RMSE

In [14]:
import numpy as np
import pandas as pd
import warnings as wn
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
wn.filterwarnings('ignore')
from sklearn.externals import joblib


rfr  = joblib.load('C:/Users/XHDMW/Desktop/ML/5001 Kaggle/msbd5001-fall2019/model_save/important model/train_model.m')
test = pd.read_csv(r'test.csv')
test['total_positive_reviews'] = test['total_positive_reviews'].apply(clean_pos)
test['total_negative_reviews'] = test['total_negative_reviews'].apply(clean_neg)
test['reviews_level'] = test['total_positive_reviews']/test['total_negative_reviews']

for item in genres_list:
    test[item] = test['genres'].apply(lambda x: 1 if item in x else 0)
    
for item in categories_list:
    test[item] = test['categories'].apply(lambda x: 1 if item in x else 0)
    
for item in tags_list:
    if item not in test:
        test[item] = test['tags'].apply(lambda x: 1 if item in x else 0)
    else:
        for index, row in test.iterrows():
            if row[item] != 1:
                if item in row['tags']:
                    row[item] = 1
                else:
                    row[item] = 0
            else:
                row[item] = 1
# 根据genres对应的平均时长评分                
test['genres_score'] = test['genres'].apply(computeGenScore)
# 根据categories对应的平均时长评分  
test['categories_score'] = test['categories'].apply(computeCateScore)
# 根据tags对应的平均时长评分  
test['tags_score'] = test['tags'].apply(computeTagScore)
# 是否有色情，暴力，血腥等成人元素
def is_adult(x):
    if 'Nudity' in x \
    or 'Sexual Content' in x \
    or 'Gore' in x\
    or 'Violent' in x:
        return 1
    else:
        return 0
    
test['adult_con'] = test['genres'].apply(is_adult)
test['adult_con'].value_counts()

                
# 根据评论反映游戏热度
def is_hot(x):
    if x >= 28000:
        return 2
    elif x >= 10000:
        return 1
    else:
        return 0

test['comment_total'] = test['total_positive_reviews'] + test['total_negative_reviews']
test['hot_level'] = test['comment_total'].apply(is_hot)

# final_test = pd.concat([\
#                          test[one_hot_col], \
#                          test[['total_positive_reviews', 'total_negative_reviews', 'reviews_level', 'is_free','price', \
#                               'comment_total', 'hot_level', 'adult_con']]\
#                         ], \
#                          axis=1, join='inner')
test['purchase_date_full_clean'] = test['purchase_date'].apply(full_clean_buy_date)
test['purchase_date_clean'] = test['purchase_date'].apply(clean_buy_date)
test['release_date_clean'] = test['release_date'].apply(clean_realease_date)
test['purchase_year'] = test['purchase_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[0]))
test['purchase_month'] = test['purchase_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[1]))
test['release_year'] = test['release_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[0]))
test['release_month'] = test['release_date_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[1]))
test['purchase_day'] = test['purchase_date_full_clean'].apply(lambda x: -1 if x == -1 else int(x.split('-')[2]))
print(time_mean, cnt_mean)

def month_total_play_time(x):
    if x in total_play_time_map:
        return total_play_time_map[x]
    else:
        return time_mean
    
def month_total_play_time(x):
    if x in total_play_time_map:
        return total_play_cnt_map[x]
    else:
        return cnt_mean

test['month_total_play_time'] = test['purchase_date_clean'].apply(month_total_play_time)
test['month_total_play_cnt'] = test['purchase_date_clean'].apply(month_total_play_cnt)

final_test = pd.concat([\
                         test[one_hot_col],\
                         test[['total_positive_reviews', 'total_negative_reviews', 'reviews_level', 'is_free',\
                                'comment_total', 'hot_level', 'price', 'adult_con',\
                                'month_total_play_time', 'month_total_play_cnt', 'purchase_year', 'purchase_month',\
                                'release_year', 'release_month', 'genres_score', 'categories_score', 'tags_score',\
                                ]]],\
                       axis=1, join='inner')
# final_test = test[['total_positive_reviews', 'total_negative_reviews', 'reviews_level', 'is_free',\
#                     'comment_total', 'hot_level', 'price', 'adult_con', \
#                     'month_total_play_time', 'month_total_play_cnt', 'purchase_year', 'purchase_month',\
#                     'release_year', 'release_month', 'genres_score', 'categories_score', 'tags_score']]
test_x = final_test.as_matrix()
test['playtime_forever'] = rfr.predict(test_x)

samplesubmission = pd.read_csv(r'samplesubmission.csv')
samplesubmission['playtime_forever'] = test['playtime_forever']
samplesubmission.to_csv('samplesubmission.csv', index=False)

28.552991452991453 9.153846153846153
